# 2. 온라인 선호지수 개발

## 2.2 협업 필터링을 통한 선호도 점수 행렬 도출

In [2]:
import numpy as np
import pandas as pd
import sklearn
from pandas import Series, DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [2]:
#데이터가져오기(기본 데이터)
Product = pd.read_csv("C:/Users/lsh93/Desktop/data/Product.csv")
Search1 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search1.csv")
Search2 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search2.csv")
Custom = pd.read_csv("C:/Users/lsh93/Desktop/data/Custom.csv")
Session = pd.read_csv("C:/Users/lsh93/Desktop/data/Session.csv")
Master = pd.read_csv("C:/Users/lsh93/Desktop/data/Master.csv")

C:\Users\lsh93\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
#데이터가져오기(created from 'Developing Online Preference 1')
Client_Behavior = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Client_Behavior.csv")
Product_Used = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Product_Used.csv")
Client_Used =  pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Client_Used.csv")
Custom_Used = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Custom_Used.csv")
Search1_Used = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Search1_Used.csv")
Session_Used = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Session_Used.csv")

C:\Users\lsh93\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
#앞에 unnamed가 붙여진 건 삭제해줘야 할듯,, 자동으로 index column붙은 듯 
#데이터 보기
Client_Behavior[0:1]

,Unnamed: 0,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,...,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,KWD_NM,SEARCH_CNT
0,0,4148106,6662803,94,570603,색상:WHT0_WHT0-(WHITE)|사이즈:115 / 1개,데상트,"39,000",1,55,...,"1,371",mobile,Seoul,Seoul,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,라코스떼 남성티셔츠,2


In [18]:
Product_Used[0:1]

,Unnamed: 0,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,0,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,데상트,"39,000",2


In [19]:
Client_Used[0:1]

,Unnamed: 0,CLNT_ID
0,87,2268789


In [20]:
Custom_Used[0:1]

,Unnamed: 0,CLNT_ID,CLNT_GENDER,CLNT_AGE
0,0,4830726,F,30


In [21]:
Search1_Used[0:1]

,Unnamed: 0,CLNT_ID,SESS_ID,KWD_NM,SEARCH_CNT
0,0,5607714,7112876,빌리프 아이크림,6


In [22]:
Session_Used[0:1]

,Unnamed: 0,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
0,0,5873599,8641867,9,20180509,82.0,"1,890",mobile,Daejeon,Daejeon


In [23]:
#첫번째 column 모두 삭제
Client_Behavior = Client_Behavior.drop(Client_Behavior.columns[[0]], axis=1)
Product_Used = Product_Used.drop(Product_Used.columns[[0]], axis=1)
Client_Used = Client_Used.drop(Client_Used.columns[[0]], axis=1)
Custom_Used = Custom_Used.drop(Custom_Used.columns[[0]], axis=1)
Search1_Used = Search1_Used.drop(Search1_Used.columns[[0]], axis=1)
Session_Used = Session_Used.drop(Session_Used.columns[[0]], axis=1)

In [26]:
print(Client_Behavior.columns.values[0])
print(Product_Used.columns.values[0])
print(Client_Used.columns.values[0])
print(Custom_Used.columns.values[0])
print(Search1_Used.columns.values[0])
print(Session_Used.columns.values[0])

CLNT_ID
CLNT_ID
CLNT_ID
CLNT_ID
CLNT_ID
CLNT_ID


In [125]:
#데이터내보내기
Client_Behavior.to_csv("C:/Users/lsh93/Desktop/data/UBCF2/Client_Behavior.csv", encoding="UTF-8")
Product_Used.to_csv("C:/Users/lsh93/Desktop/data/UBCF2/Product_Used.csv", encoding="UTF-8")
Client_Used.to_csv("C:/Users/lsh93/Desktop/data/UBCF2/Client_Used.csv", encoding="UTF-8")
Custom_Used.to_csv("C:/Users/lsh93/Desktop/data/UBCF2/Custom_Used.csv", encoding="UTF-8")
Search1_Used.to_csv("C:/Users/lsh93/Desktop/data/UBCF2/Search1_Used.csv", encoding="UTF-8")
Session_Used.to_csv("C:/Users/lsh93/Desktop/data/UBCF2/Sesion_Used.csv", encoding="UTF-8")

### 1) 구매 count를 이용한 온라인 선호지수 생성

In [3]:
#데이터 불러오기
Purchase_Count = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Purchase_Count.csv")

In [4]:
Purchase_Count.head()

,Unnamed: 0,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
0,4140076,23.0,52.0,17.0,5.0,5.0,24.0,8.0,8.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4148106,3.0,3.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5298612,1.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5553084,7.0,13.0,21.0,0.0,3.0,10.0,11.0,19.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5639922,11.0,16.0,9.0,0.0,0.0,17.0,4.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0


In [5]:
Purchase_Count = Purchase_Count.rename(columns = {'Unnamed: 0':'CLNT_ID'})

In [6]:
Purchase_Count.head()

,CLNT_ID,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
0,4140076,23.0,52.0,17.0,5.0,5.0,24.0,8.0,8.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4148106,3.0,3.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5298612,1.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5553084,7.0,13.0,21.0,0.0,3.0,10.0,11.0,19.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5639922,11.0,16.0,9.0,0.0,0.0,17.0,4.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0


In [7]:
row_names = Purchase_Count['CLNT_ID'].as_matrix()
row_names

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([4140076, 4148106, 5298612, ..., 4362431, 6510160, 1835621],
      dtype=int64)

In [8]:
col_names = Purchase_Count.columns.values
col_names = col_names[1:]

In [9]:
temp_matrix = Purchase_Count.as_matrix()

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [10]:
temp_matrix = temp_matrix[:,1:]

In [11]:
temp_matrix.shape

(375839, 37)

In [12]:
temp = pd.DataFrame(temp_matrix, columns = col_names, index = row_names)

In [13]:
#일부 row만 출력
temp[0:15] #0번째 row부터 14번째 row까지 출력됨 

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
4140076,23.0,52.0,17.0,5.0,5.0,24.0,8.0,8.0,7.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4148106,3.0,3.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5298612,1.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5553084,7.0,13.0,21.0,0.0,3.0,10.0,11.0,19.0,0.0,6.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5639922,11.0,16.0,9.0,0.0,0.0,17.0,4.0,55.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
5650295,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5851368,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4128075,10.0,0.0,0.0,0.0,0.0,1.0,8.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4207528,3.0,2.0,1.0,2.0,0.0,7.0,0.0,5.0,2.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4219875,5.0,0.0,0.0,6.0,0.0,2.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#이제 이 Purchase_Count를 이용할 것임
Purchase_Count = temp
print(Purchase_Count.head())
print(Purchase_Count.shape)

         스포츠패션  남성의류  유아동의류  청소/세탁/욕실용품   완구  패션잡화  시즌스포츠  여성의류  인테리어/조명  \
4140076   23.0  52.0   17.0         5.0  5.0  24.0    8.0   8.0      7.0   
4148106    3.0   3.0    0.0         0.0  0.0   9.0    7.0   0.0      0.0   
5298612    1.0   4.0    2.0         0.0  0.0   0.0    0.0   0.0      0.0   
5553084    7.0  13.0   21.0         0.0  3.0  10.0   11.0  19.0      0.0   
5639922   11.0  16.0    9.0         0.0  0.0  17.0    4.0  55.0      0.0   

         속옷/양말/홈웨어   ...      음료  구기/필드스포츠  냉장/세탁가전  냉동식품  냉장식품  원예/애완  상품권  \
4140076        4.0   ...     0.0       0.0      0.0   0.0   0.0    0.0  0.0   
4148106        0.0   ...     0.0       0.0      0.0   0.0   0.0    0.0  0.0   
5298612        0.0   ...     0.0       0.0      0.0   0.0   0.0    0.0  0.0   
5553084        6.0   ...     0.0       3.0      0.0   0.0   0.0    0.0  0.0   
5639922       12.0   ...     0.0       0.0      0.0   0.0   0.0   16.0  0.0   

         자동차용품  축산물  영상/음향가전  
4140076    0.0  0.0      0.0  
414810

In [15]:
#안에 있는 element값만 가져온 것
print(temp_matrix)
print(temp_matrix.shape)

[[23. 52. 17. ...  0.  0.  0.]
 [ 3.  3.  0. ...  0.  0.  0.]
 [ 1.  4.  2. ...  0.  0.  0.]
 ...
 [ 2.  0.  0. ...  0.  0.  0.]
 [ 0.  1.  0. ...  0.  0.  0.]
 [ 2.  0.  0. ...  0.  0.  0.]]
(375839, 37)


In [16]:
#PC라는 명칭으로 이용할 것
PC = temp_matrix

In [17]:
PC

array([[23., 52., 17., ...,  0.,  0.,  0.],
       [ 3.,  3.,  0., ...,  0.,  0.,  0.],
       [ 1.,  4.,  2., ...,  0.,  0.,  0.],
       ...,
       [ 2.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 2.,  0.,  0., ...,  0.,  0.,  0.]])

In [18]:
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))

Sparsity: {:4.2f}% 7.940173922948462


In [19]:
from sklearn.model_selection import train_test_split

In [31]:
#test set과 training set으로 나누기
PC_train, PC_test = train_test_split(Purchase_Count, test_size = 0.33, random_state = 50)

In [32]:
#print(PC_train)
print(PC_train.shape)
#print(PC_test)
print(PC_test.shape)

(251812, 37)
(124027, 37)


In [33]:
PC_train = pd.DataFrame(PC_train)
PC_train.head()

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
598339,8.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1386382,4.0,1.0,5.0,0.0,0.0,1.0,7.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5853909,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5256290,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
PC_test = pd.DataFrame(PC_test)
PC_train.head()

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
598339,8.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1386382,4.0,1.0,5.0,0.0,0.0,1.0,7.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5853909,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5256290,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Cosine Distance를 이용한 선호점수 계산

In [35]:
#row 길이가 너무 커서 줄일 필요가 있음 (나중에 클러스터링 결과에서 랜덤추출할 예정)
temp_train = PC_train[0:1000]
temp_test = PC_test[0:330]

In [36]:
temp_train.head()
#temp_test.head()

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
598339,8.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1386382,4.0,1.0,5.0,0.0,0.0,1.0,7.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5853909,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5256290,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
#array 형태로 변환
temp_train_mat = temp_train.as_matrix()
temp_test_mat = temp_test.as_matrix()

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
temp_train_mat.shape

(1000, 37)

In [41]:
temp_test_mat.shape

(330, 37)

####  Cosine Similarity 계산

In [42]:
#코사인 유사도 계산 
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [43]:
dist_out = 1 - pairwise_distances(temp_train_mat, metric="cosine")

In [44]:
dist_out 

array([[1.        , 0.04358136, 0.38964803, ..., 0.26501546, 0.01544551,
        0.74074074],
       [0.04358136, 1.        , 0.02022783, ..., 0.14617634, 0.        ,
        0.        ],
       [0.38964803, 0.02022783, 1.        , ..., 0.1537552 , 0.55917211,
        0.27504567],
       ...,
       [0.26501546, 0.14617634, 0.1537552 , ..., 1.        , 0.41444658,
        0.2981424 ],
       [0.01544551, 0.        , 0.55917211, ..., 0.41444658, 1.        ,
        0.04633654],
       [0.74074074, 0.        , 0.27504567, ..., 0.2981424 , 0.04633654,
        1.        ]])

In [45]:
dist_out.shape

(1000, 1000)

In [46]:
#Client의 알려지지 않은 평가 예측 
#거리행렬과 평가행렬의 내적을 구하고 평가수의 데이터를 정규화해 계산
user_pred = dist_out.dot(temp_train_mat)/np.array([np.abs(dist_out).sum(axis = 1)]).T

In [47]:
from sklearn.metrics import mean_squared_error
def get_mse(pred,actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    
    return mean_squared_error(pred, actual)

In [48]:
#모델의 예측오차율
get_mse(user_pred, temp_train_mat)

14.42691745539262

In [49]:
#테스트 데이터를 사용해 정확도를 확인해보기
get_mse(user_pred, temp_test_mat)

15.689135306586335

In [50]:
from sklearn.neighbors import NearestNeighbors

In [51]:
k = 5
neigh = NearestNeighbors(k, 'cosine')

In [52]:
neigh.fit(temp_train_mat)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2,
         radius='cosine')

In [53]:
#각 사용자별로 가장 유사한 5명의 사용자 찾고 유사도 계산
top_k_distances, top_k_users = neigh.kneighbors(temp_train_mat, return_distance = True)

In [54]:
top_k_distances.shape

(1000, 5)

In [55]:
top_k_users.shape

(1000, 5)

In [56]:
top_k_users[4]

array([  4, 598,  46, 934, 302], dtype=int64)

In [57]:
#훈련 데이터의 알려지지 않은 구매 예측
user_pred_k = np.zeros(temp_train_mat.shape)

In [58]:
for i in range(temp_train.shape[0]):
    user_pred_k[i,:] = top_k_distances[i].T.dot(temp_train_mat[top_k_users][i]) / np.array([np.abs(top_k_distances[i].T).sum(axis = 0)]).T  

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [59]:
user_pred_k.shape

(1000, 37)

In [60]:
user_pred_k

array([[6.46672054, 0.44546196, 0.26974061, ..., 0.        , 0.        ,
        0.        ],
       [0.73768794, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [2.29264123, 0.        , 2.71458165, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.27806715, 0.48532617, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.22966307, ..., 0.        , 0.        ,
        0.        ],
       [1.46049571, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [61]:
#nan값 있어서 mse계산할 수 없음. 
#왜 nan값이 나올까,,
online_pref = pd.DataFrame(user_pred_k)
online_pref[0:10]

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,6.466721,0.445462,0.269741,0.0,0.0,0.000000,0.000000,0.890924,0.0,0.000000,...,0.445462,0.668193,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.737688,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.252130,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
2,2.292641,0.000000,2.714582,0.0,0.0,0.745085,3.119941,0.289564,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.521164,0.196982,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
5,0.333333,0.000000,0.000000,0.0,0.0,0.333333,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
6,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,4.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.000000,0.000000,0.000000,0.0,0.0,0.225511,0.000000,0.000000,0.0,0.473035,...,0.000000,0.000000,0.0,7.220993,0.0,0.676532,0.0,0.0,0.0,0.316251
9,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.750000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000


- nan가 들어간 row 전체를 제거한다. 대체하는 건 근거가 없으므로

In [62]:
#nan가 들어간 element의 row의 인덱스 반환
na_element_index = np.where(np.isnan(online_pref))
na_element_index

(array([  7,   7,   7, ..., 996, 996, 996], dtype=int64),
 array([ 0,  1,  2, ..., 34, 35, 36], dtype=int64))

In [88]:
#nan가 들어간 row의 인덱스 반환
na_row_index = np.unique(na_element_index[0])
na_row_index

array([  7,  22,  25,  28,  29,  37,  41,  43,  44,  53,  56,  57,  75,
        78,  80,  87,  90,  91, 105, 111, 112, 117, 123, 125, 137, 143,
       149, 160, 164, 171, 175, 179, 192, 196, 205, 220, 228, 229, 232,
       233, 236, 237, 238, 242, 243, 246, 258, 259, 266, 268, 273, 276,
       282, 286, 287, 293, 306, 323, 333, 347, 354, 357, 358, 364, 367,
       368, 371, 388, 392, 394, 405, 407, 409, 422, 424, 439, 454, 466,
       468, 469, 472, 484, 502, 505, 511, 513, 514, 515, 516, 525, 526,
       529, 535, 541, 543, 546, 547, 556, 559, 561, 562, 569, 570, 574,
       575, 588, 601, 604, 607, 608, 612, 617, 623, 640, 644, 648, 650,
       651, 656, 658, 663, 664, 665, 679, 690, 702, 703, 705, 717, 718,
       732, 733, 742, 744, 756, 764, 775, 776, 778, 780, 785, 789, 793,
       800, 804, 807, 810, 820, 830, 833, 842, 849, 857, 863, 868, 872,
       889, 892, 902, 906, 911, 921, 923, 956, 958, 959, 962, 964, 967,
       981, 983, 989, 996], dtype=int64)

In [64]:
#제거할 client id의 목록
#temp_train 데이터프레임에서 반환
removed_CLNT_ID = temp_train.index.values[na_row_index]
removed_CLNT_ID

array([2155053, 5311863, 1371482, 2312035, 1332223, 2735178, 1173536,
       3683299,  505740, 6569904,  568937, 1058051, 2312101, 6345603,
       6532328, 1508366, 2713949, 5675404,  106734, 4494904, 6434825,
       2584803, 4026061, 6118473, 1750178, 2344413, 4989404,  155573,
       3402137, 3464107,  964886, 1712329, 4744206, 4313519,  320684,
       2457572, 5131019,  958213, 6471883,  359744, 5737406,  431106,
       5227723, 3043938, 2535392, 2874954, 5910696, 1391773,  266685,
       4243335, 5166447,   27688, 6419370, 1808260, 3760070, 3886405,
       6438936, 1226700, 2976874, 4634106, 4056121, 4498332, 2900325,
       1981334, 1071844, 4029836, 5192347, 6561377, 5873001, 1804018,
       1398740, 2392853, 4935201, 5599436, 2143681, 2976316,   36701,
       3682644, 2124156, 3636086, 3695526, 6569661, 4934309, 4439538,
       4526521, 1176151, 4636041, 5522676, 6397479, 3394654, 5727435,
       4881270, 3582620,  161741, 1494854, 4656469, 1723308, 3165394,
       2648052, 4304

In [65]:
#제거하지 않고 남는 Client id의 목록
existed_CLNT_ID = np.delete(temp_train.index.values,na_row_index)
existed_CLNT_ID.shape

(827,)

In [82]:
#이제 online_pref에서 nan가 들어간 row 제거
online_pref2 = online_pref.dropna()
online_pref2.shape

(827, 37)

In [72]:
#결측치가 제거된 dataframe을 array로 바꾼다
user_pred_k2 = online_pref2.as_matrix()
user_pred_k2.shape

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


(827, 37)

In [73]:
#결측치 row제거된 선호지수행렬 
user_pred_k2

array([[6.46672054, 0.44546196, 0.26974061, ..., 0.        , 0.        ,
        0.        ],
       [0.73768794, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [2.29264123, 0.        , 2.71458165, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.27806715, 0.48532617, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.22966307, ..., 0.        , 0.        ,
        0.        ],
       [1.46049571, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [84]:
temp_train_mat2 = temp_train_mat

In [86]:
na_row_index

array([  7,  22,  25,  28,  29,  37,  41,  43,  44,  53,  56,  57,  75,
        78,  80,  87,  90,  91, 105, 111, 112, 117, 123, 125, 137, 143,
       149, 160, 164, 171, 175, 179, 192, 196, 205, 220, 228, 229, 232,
       233, 236, 237, 238, 242, 243, 246, 258, 259, 266, 268, 273, 276,
       282, 286, 287, 293, 306, 323, 333, 347, 354, 357, 358, 364, 367,
       368, 371, 388, 392, 394, 405, 407, 409, 422, 424, 439, 454, 466,
       468, 469, 472, 484, 502, 505, 511, 513, 514, 515, 516, 525, 526,
       529, 535, 541, 543, 546, 547, 556, 559, 561, 562, 569, 570, 574,
       575, 588, 601, 604, 607, 608, 612, 617, 623, 640, 644, 648, 650,
       651, 656, 658, 663, 664, 665, 679, 690, 702, 703, 705, 717, 718,
       732, 733, 742, 744, 756, 764, 775, 776, 778, 780, 785, 789, 793,
       800, 804, 807, 810, 820, 830, 833, 842, 849, 857, 863, 868, 872,
       889, 892, 902, 906, 911, 921, 923, 956, 958, 959, 962, 964, 967,
       981, 983, 989, 996], dtype=int64)

In [92]:
#본래 training data에서도 nan이 나온 index를 제외해준다
exist_idx = np.delete([range(1000)],na_row_index)
temp_train_mat2 = temp_train_mat2[exist_idx,:]
temp_train_mat2

array([[8., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 1., 5., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 6., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.]])

In [93]:
#모델의 개선 여부 확인 -> 개선됨! 
get_mse(user_pred_k2, temp_train_mat2)

5.71507811114122

In [94]:
get_mse(user_pred_k2, temp_test_mat)

18.862746442249264

- 선호점수 matrix 생성 (index, column 모두 달아서)

In [95]:
user_pred_k2 

array([[6.46672054, 0.44546196, 0.26974061, ..., 0.        , 0.        ,
        0.        ],
       [0.73768794, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [2.29264123, 0.        , 2.71458165, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.27806715, 0.48532617, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.22966307, ..., 0.        , 0.        ,
        0.        ],
       [1.46049571, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [251]:
#제거하지 않고 남은 index의 목록
existed_CLNT_ID

array([ 501089, 6461629, 2794743,  619336,   96115, 4551115, 2705960,
       1501046,    6001, 4878190, 4433077, 2318507, 5477642,  190156,
       3918701, 5215589, 2663610,  757041, 1404401, 4019725, 1964806,
       5864542, 3442990, 2516201, 6264193, 2119272, 6352888,  298521,
       6521459,  313042,  358619, 3169768, 5500215, 2005300,  668749,
       6022193, 3911435, 5997403, 5363818, 6043390, 2611936, 3444206,
       2450155, 3633855, 6354661, 4032123,   29626, 4006183, 5107918,
       2798238,  560109, 3599202,  570574, 4158386,   47058,  937344,
       4867494, 4976377,  811650, 3588525, 1563614, 5253045, 6533222,
       3225617,   83617, 2930297, 2094969, 2233347, 4002018, 6295576,
       3011468, 3818126, 5853476, 1845000, 4675457,  871034, 5852553,
       3911422, 3919219, 4782003, 4449151,  917618, 2936747,  939196,
       6023403, 4249157, 2539268, 1087931, 3808803, 1127823, 5682069,
       6214392, 5990691, 6211363, 3122712, 2600985, 6524914, 6167312,
       2140836, 6068

In [96]:
col_names = Purchase_Count.columns.values
row_names = existed_CLNT_ID
print(col_names.shape)
print(row_names.shape)
print(user_pred_k2.shape)

(37,)
(827,)
(827, 37)


In [100]:
Online_Preference_Count = pd.DataFrame(user_pred_k2, columns = col_names, index = row_names)

In [101]:
Online_Preference_Count.head()

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
598339,6.466721,0.445462,0.269741,0.0,0.0,0.000000,0.000000,0.890924,0.0,0.0,...,0.445462,0.668193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000521,0.737688,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.252130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1386382,2.292641,0.000000,2.714582,0.0,0.0,0.745085,3.119941,0.289564,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5853909,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.521164,0.196982,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5256290,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
Existed_Client1 = pd.DataFrame(Online_Preference_Count.index.values, columns = ['CLNT_ID'])
Existed_Client1.head()

,CLNT_ID
0,598339
1,1000521
2,1386382
3,5853909
4,5256290


In [114]:
#데이터 내보내기
Online_Preference_Count.to_csv("C:/Users/lsh93/Desktop/data/UBCF2/Online_Preference_Count.csv", encoding="UTF-8")
Existed_Client1.to_csv("C:/Users/lsh93/Desktop/data/UBCF2/Existed_Client1.csv", encoding="UTF-8")

2) 검색 횟수를 이용한 온라인 선호지수 생성